# Obtención de datos

Para obtener los datos vamos a usar las descripciones en cada video de las mañaneras. Pues nos provee de un resumen del video del día. De esta manera ya no tenemos que lidiar con conversaciones del presidente cuando usamos las transcripciones.

## Importaciones

In [497]:
import json 
from datetime import datetime
import re

import pandas as pd 
import numpy as np

from googleapiclient.errors import HttpError
from Google import Create_Service

## Llamamiento a la API de YouTube

<code>retrieve_playlists_items</code> Función que hace la petición a la API de youtube. Se le da un objeto <code>service</code> que es la información sobre el tipo de API y la autenticación. También recibe <code>playlistId</code> que es el ID de la playlist que queremos.

In [498]:
def retrieve_playlists_items(service, playlistId):
    items =[]
    try:
        response = service.playlistItems().list(
            part = 'contentDetails, snippet, status',
            playlistId = playlistId,
            maxResults = 5
        ).execute()

        items.extend(response.get('items'))
        nextPageToken = response.get('nextPageToken')

        while nextPageToken:
            response = service.playlistItems().list(
                part = 'contentDetails, snippet, status',
                playlistId = playlistId,
                maxResults = 5,
                pageToken = nextPageToken
            ).execute()

            items.extend(response.get('items'))
            nextPageToken = response.get('nextPageToken')

        return items

    except HttpError as e:
        errMsg = json.loads(e.content)
        print("HTTP Error: ")
        print(errMsg['error']['message'])
        return []

Creando constantes para llamar a la API. Toda esta es información sobre el tipo de API y los archivos necesarios para hacer la conexión con Google Cloud

In [499]:
CLIENT_SECRETS_FILE = 'client_secret.json'  # Archivo .json que se obtiene al crear las credenciales OAuth en https://console.cloud.google.com/
API_NAME = 'youtube'
API_VERSION = 'v3'
SCOPES = ['https://www.googleapis.com/auth/youtube.readonly']

service = Create_Service(CLIENT_SECRETS_FILE, API_NAME, API_VERSION, SCOPES)

client_secret.json-youtube-v3-(['https://www.googleapis.com/auth/youtube.readonly'],)
['https://www.googleapis.com/auth/youtube.readonly']
youtube service created successfully


El objeto <code>service</code> nos da accesos a todos los recursos de la API

In [500]:
dir(service)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_basic_methods',
 '_add_nested_resources',
 '_add_next_methods',
 '_baseUrl',
 '_developerKey',
 '_dynamic_attrs',
 '_http',
 '_model',
 '_requestBuilder',
 '_resourceDesc',
 '_rootDesc',
 '_schema',
 '_set_dynamic_attr',
 '_set_service_methods',
 'abuseReports',
 'activities',
 'captions',
 'channelBanners',
 'channelSections',
 'channels',
 'close',
 'commentThreads',
 'comments',
 'i18nLanguages',
 'i18nRegions',
 'liveBroadcasts',
 'liveChatBans',
 'liveChatMessages',
 'liveChatModerators',
 'liveStreams',
 'members',
 'membershipsLevels',
 'new_batch_h

Hacemos una llamada a a función de <code>retrieve_playlists_items</code> para obtener todos los videos una playlist especifica. En este caso, los videos de la mañaneras

In [501]:
playlistId = 'PLRnlRGar-_296KTsVL0R6MEbpwJzD8ppA' # id de la playlist de las mañaneras
playlists_items = retrieve_playlists_items(service, playlistId)

In [502]:
playlists_items[0] # El último video agregado a la playlist

{'kind': 'youtube#playlistItem',
 'etag': '3xCC5OTcxHdoj5hmgSVw5mNkh8Q',
 'id': 'UExSbmxSR2FyLV8yOTZLVHNWTDBSNk1FYnB3SnpEOHBwQS41OThGNjE5NEFFRURFREE3',
 'snippet': {'publishedAt': '2023-05-12T03:54:24Z',
  'channelId': 'UCxEgOKuI-n-WOJaNcisHvSg',
  'title': 'Conferencia de prensa en vivo, desde Palacio Nacional. Viernes 12 de mayo 2023 | Presidente AMLO',
  'description': 'Conferencia de prensa matutina, desde Palacio Nacional. Viernes 12 de mayo 2023 | Presidente AMLO.\n\nSigue las actividades del presidente de México:\n\nSitio web:\nhttp://presidente.gob.mx\n\nYouTube:\nhttps://www.youtube.com/lopezobrador\n\nFacebook:\nhttps://facebook.com/lopezobrador.org.mx\n\nTwitter:\nhttps://twitter.com/lopezobrador_\n\nInstagram:\nhttps://www.instagram.com/lopezobrador/\n\nTelegram: \nhttps://t.me/PresidenteAMLO\n\nSpotify:\nhttps://spoti.fi/3QocikN\n\n#EnVivo #AMLO',
  'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/nzc-ZmovlR0/default_live.jpg',
    'width': 120,
    'height': 90},

## Guardando los datos en un DataFrame

Los datos están guardados en un json. Podemos obtener cada elemento de la variable utilizando las posiciones.

In [503]:
playlists_items[0]['snippet']

{'publishedAt': '2023-05-12T03:54:24Z',
 'channelId': 'UCxEgOKuI-n-WOJaNcisHvSg',
 'title': 'Conferencia de prensa en vivo, desde Palacio Nacional. Viernes 12 de mayo 2023 | Presidente AMLO',
 'description': 'Conferencia de prensa matutina, desde Palacio Nacional. Viernes 12 de mayo 2023 | Presidente AMLO.\n\nSigue las actividades del presidente de México:\n\nSitio web:\nhttp://presidente.gob.mx\n\nYouTube:\nhttps://www.youtube.com/lopezobrador\n\nFacebook:\nhttps://facebook.com/lopezobrador.org.mx\n\nTwitter:\nhttps://twitter.com/lopezobrador_\n\nInstagram:\nhttps://www.instagram.com/lopezobrador/\n\nTelegram: \nhttps://t.me/PresidenteAMLO\n\nSpotify:\nhttps://spoti.fi/3QocikN\n\n#EnVivo #AMLO',
 'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/nzc-ZmovlR0/default_live.jpg',
   'width': 120,
   'height': 90},
  'medium': {'url': 'https://i.ytimg.com/vi/nzc-ZmovlR0/mqdefault_live.jpg',
   'width': 320,
   'height': 180},
  'high': {'url': 'https://i.ytimg.com/vi/nzc-ZmovlR0/hq

Extraeremos sólo la información necesaria para hacer el entrenamiento. Estos datos son: título, descripción y la fecha de publicación

In [504]:

data = []

for item in playlists_items:
    snippet = item['snippet']
    published_at = snippet['publishedAt']
    title = snippet['title']
    description = snippet['description']
    
    data.append([published_at, title, description])
    
df = pd.DataFrame(data, columns=['published_at', 'title', 'description'])
df = df.head(1099)

print(df)


              published_at                                              title   
0     2023-05-12T03:54:24Z  Conferencia de prensa en vivo, desde Palacio N...  \
1     2023-05-11T12:44:34Z  Sueldos de magistrados y jueces deben bajar pa...   
2     2023-05-10T12:50:23Z  Homenaje a las madres de México en su día. Con...   
3     2023-05-09T12:47:41Z  Fin de la emergencia sanitaria por COVID-19 en...   
4     2023-05-08T12:45:24Z  Primer convoy del Tren Maya llegará el 8 de ju...   
...                    ...                                                ...   
1094  2019-03-08T21:59:25Z  Conferencia matutina: Avances del Gabinete de ...   
1095  2019-03-08T21:59:25Z  Hoy anunciamos la terna de candidatos a minist...   
1096  2019-03-08T21:59:25Z  Tercera conferencia de prensa matutina desde P...   
1097  2019-03-08T21:59:25Z     Diálogo con periodistas desde Palacio Nacional   
1098  2019-03-08T21:59:25Z  Primera conferencia de prensa matutina desde P...   

                           

## Una primera limpieza

Los datos que llegan de la API están contaminados con información como minutos de YouTube en formato <code>00:00</code>, links, saltos de línea <code>\n</code> y palabras específicas como nombres redes sociales o frases que no aportan nada.
Para mejor ejemplo, tomemos un elemento.

In [505]:
df['description'][500]

'Conferencia de prensa matutina, desde Palacio Nacional. Miércoles 14 de abril 2021 | Presidente AMLO.\n\n0:00 Inicio de transmisión\n20:42 Comienza la conferencia de prensa del presidente Andrés Manuel López Obrador\n\nDecidimos reforzar la vigilancia en la frontera sur del país con el objetivo de garantizar la seguridad e integridad de las y los niños migrantes. Al ordenar el flujo de personas prevenimos situaciones que ponen en riesgo la vida de los menores y actuamos en congruencia con nuestra convicción de defensa de los derechos humanos. \n\nEs muy doloroso que usen a niñas y niños porque si un adulto va acompañado de uno, tiene más posibilidades de obtener la residencia en Estados Unidos. Si detrás de esto hay traficantes que cobran por pasar indocumentados, no podemos permitirlo. \n\nLa próxima semana nos reuniremos con gobernadores de Chiapas, Tabasco, Campeche y Quintana Roo y autoridades municipales de la zona fronteriza a fin de fortalecer las acciones dedicadas al cuidado 

Declararemos una serie de funciones que utilizarán regex para limpiar esta basura.

In [506]:
def replace_patterns(row, patterns):
    text = row['description']
    for pattern, replacement in patterns.items():
        regex = re.compile(pattern, re.IGNORECASE | re.DOTALL)
        text = regex.sub(replacement, text)
    return text


In [507]:
def remove_beginning(string):
    pattern = r'Conferencia de prensa (matutina|en vivo),?( desde (\b\w+\b,?\s)?(\b\w+\b\s)?\b\w+\b)\. \b(Lunes|Martes|Miércoles|Jueves|Viernes|Sábado|Domingo)\b [0-3]?[0-9] de \b(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)\b 20[0-9][0-9]\.?'
    replace_with = ''
    return re.sub(pattern, replace_with, string)

In [508]:
patterns = {

    # Borra palabras clave dentro de la descripción
    "Sigue las actividades del presidente de México:": "",
    "\s+Más información:": "",
    "Sitio web:\s+": "",
    "YouTube:\s+": "",
    "Facebook:\s+": "",
    "Twitter:\s+": "",
    "Instagram:\s+": "",
    "Telegram:\s+": "",
    "Spotify:": "",
    "#EnVivo\s+": "",
    "#AMLO": "",
    
    # Borrar separadores
    "\|": "", 

    # Borra el inicio la mención del presidente
    "Presidente AMLO.": "",

    # Borra saltos de línea:
    "\n": " ", 

    # Borra los saltos a minutos en YouTube:
    "[0-9]?[0-9]:[0-9][0-9]": "", 

    # Borra cualquier link de internet:
    "http(s)?://[a-zA-Z0-9./?=_-]+": "",

    # Borra la el inicio de la transmisión
    "Inicio de transmisión": "",
    "Comienza la conferencia de prensa del presidente Andrés Manuel López Obrador": "",


    "  +": " "

}   

Aplicamos las funciones y vemos los resultados

In [509]:
df['description'] = df.apply(replace_patterns, patterns=patterns, axis=1).apply(remove_beginning)

In [510]:
df['description'][500]

' Decidimos reforzar la vigilancia en la frontera sur del país con el objetivo de garantizar la seguridad e integridad de las y los niños migrantes. Al ordenar el flujo de personas prevenimos situaciones que ponen en riesgo la vida de los menores y actuamos en congruencia con nuestra convicción de defensa de los derechos humanos. Es muy doloroso que usen a niñas y niños porque si un adulto va acompañado de uno, tiene más posibilidades de obtener la residencia en Estados Unidos. Si detrás de esto hay traficantes que cobran por pasar indocumentados, no podemos permitirlo. La próxima semana nos reuniremos con gobernadores de Chiapas, Tabasco, Campeche y Quintana Roo y autoridades municipales de la zona fronteriza a fin de fortalecer las acciones dedicadas al cuidado de menores en tránsito. La titular del Sistema Nacional para el Desarrollo Integral de la Familia (DIF), María del Rocío García Pérez, se encuentra en Tapachula, Chiapas, atendiendo la crisis migratoria. De acuerdo con informa

## Separación por semana

Ahora necesitamos separar por semana, es decir, juntar todas las descripciones de una semana. De esta manera cada renglón del dataframe reprensentará una semana en específico.
Lo primero que debemos hacer es limpiar el apartado de la fecha y convertirlo a datetime.

In [511]:
df['published_at']

0       2023-05-12T03:54:24Z
1       2023-05-11T12:44:34Z
2       2023-05-10T12:50:23Z
3       2023-05-09T12:47:41Z
4       2023-05-08T12:45:24Z
                ...         
1094    2019-03-08T21:59:25Z
1095    2019-03-08T21:59:25Z
1096    2019-03-08T21:59:25Z
1097    2019-03-08T21:59:25Z
1098    2019-03-08T21:59:25Z
Name: published_at, Length: 1099, dtype: object

Lo que se haces es: tomar los primeros 10 digitos de cada elemento de la columna y convertirlo al tipo datetime con la función <code>datetime.strptime</code>. Son los primeros 10 digitos porque son los que representan la fecha de ese día, no nos importan las horas, minutos o segundos así que lo descartamos

In [512]:
clean_dates = []

for dates in df['published_at']:
    clean_dates.append( datetime.strptime(dates[:10], '%Y-%m-%d') )
df['clean_dates'] = clean_dates

In [513]:
df = df.drop('published_at', axis=1) # Ahorra borramos las fechas que no nos sirven

In [514]:
df['clean_dates'] = pd.to_datetime(df['clean_dates'])

Ya tenemos los datos como fechas tipo <code>datetime</code>

In [515]:
df

,title,description,clean_dates
0,"Conferencia de prensa en vivo, desde Palacio N...",,2023-05-12
1,Sueldos de magistrados y jueces deben bajar pa...,Fomento al deporte a través del Programa de M...,2023-05-11
2,Homenaje a las madres de México en su día. Con...,Reporte sobre noticias falsas Sesión de pregu...,2023-05-10
3,Fin de la emergencia sanitaria por COVID-19 en...,:07 Anuncio del fin de la emergencia sanitari...,2023-05-09
4,Primer convoy del Tren Maya llegará el 8 de ju...,Comparativo de marcas y precios de combustibl...,2023-05-08
...,...,...,...
1094,Conferencia matutina: Avances del Gabinete de ...,Hoy destacamos la conformación de un sistema ...,2019-03-08
1095,Hoy anunciamos la terna de candidatos a minist...,Cuarta conferencia de prensa matutina desde el...,2019-03-08
1096,Tercera conferencia de prensa matutina desde P...,Tercera conferencia de prensa matutina desde e...,2019-03-08
1097,Diálogo con periodistas desde Palacio Nacional,Segunda conferencia de prensa matutina del pre...,2019-03-08


In [516]:
df['clean_dates']

0      2023-05-12
1      2023-05-11
2      2023-05-10
3      2023-05-09
4      2023-05-08
          ...    
1094   2019-03-08
1095   2019-03-08
1096   2019-03-08
1097   2019-03-08
1098   2019-03-08
Name: clean_dates, Length: 1099, dtype: datetime64[ns]

La mejor manera de manipular datos con fechas es creando una serie de tiempo, para estos se utiliza la libería <code>datetime</code>. Convertimos el Data Frame en una serie de tiempo con <code>pd.set_index</code>.
De esta manera las fechas ahora serán los índices de el dataframe

In [517]:
df.set_index('clean_dates', inplace=True)

In [518]:
df

,title,description
clean_dates,,
2023-05-12,"Conferencia de prensa en vivo, desde Palacio N...",
2023-05-11,Sueldos de magistrados y jueces deben bajar pa...,Fomento al deporte a través del Programa de M...
2023-05-10,Homenaje a las madres de México en su día. Con...,Reporte sobre noticias falsas Sesión de pregu...
2023-05-09,Fin de la emergencia sanitaria por COVID-19 en...,:07 Anuncio del fin de la emergencia sanitari...
2023-05-08,Primer convoy del Tren Maya llegará el 8 de ju...,Comparativo de marcas y precios de combustibl...
...,...,...
2019-03-08,Conferencia matutina: Avances del Gabinete de ...,Hoy destacamos la conformación de un sistema ...
2019-03-08,Hoy anunciamos la terna de candidatos a minist...,Cuarta conferencia de prensa matutina desde el...
2019-03-08,Tercera conferencia de prensa matutina desde P...,Tercera conferencia de prensa matutina desde e...


In [519]:
df['description'].to_csv('dataframe.csv')

In [520]:
df.resample('W').sum()

,title,description
clean_dates,,
2019-03-10,Presidente AMLO llama a terminar con feminicid...,Hoy conmemoramos la lucha de las mujeres por ...
2019-03-17,Reformas constitucionales aprobadas representa...,Hoy dimos a conocer el análisis de la recient...
2019-03-24,Presidente firma compromiso de no reelección; ...,Hoy dimos a conocer que ya firmamos la carta ...
2019-03-31,Acciones para mejorar protección a defensores ...,Hoy informamos sobre las acciones para mejora...
2019-04-07,Avanza entrega de apoyos de 'Programas Integra...,En el Gobierno de México avanzamos con entusi...
...,...,...
2023-04-16,Avanza instalación de doble vía electrificada ...,Comparativo de marcas y precios de combustibl...
2023-04-23,Avance mayor al 60 por ciento en obras del tra...,Comparativo de marcas y precios de combustibl...
2023-04-30,Conferencia de prensa matutina desde Palacio N...,Comparativo de marcas y precios de combustibl...


Ahora le decimos que separe, con respecto a las fechas, semanalmente. De esta manera va a concatenar los contenidos de las descripciones y los títulos.

In [521]:
weekly_ts = df.resample('W').sum()

Se muestra el dataframe

In [522]:
weekly_ts

,title,description
clean_dates,,
2019-03-10,Presidente AMLO llama a terminar con feminicid...,Hoy conmemoramos la lucha de las mujeres por ...
2019-03-17,Reformas constitucionales aprobadas representa...,Hoy dimos a conocer el análisis de la recient...
2019-03-24,Presidente firma compromiso de no reelección; ...,Hoy dimos a conocer que ya firmamos la carta ...
2019-03-31,Acciones para mejorar protección a defensores ...,Hoy informamos sobre las acciones para mejora...
2019-04-07,Avanza entrega de apoyos de 'Programas Integra...,En el Gobierno de México avanzamos con entusi...
...,...,...
2023-04-16,Avanza instalación de doble vía electrificada ...,Comparativo de marcas y precios de combustibl...
2023-04-23,Avance mayor al 60 por ciento en obras del tra...,Comparativo de marcas y precios de combustibl...
2023-04-30,Conferencia de prensa matutina desde Palacio N...,Comparativo de marcas y precios de combustibl...


Ahora se guarda el dataframe en un .txt para que el preprocesamiento pueda llamarlo y utlizarlo

In [523]:
weekly_ts['description'].to_csv('mañaneras.csv')

Así es como se termina la obtención de datos, ya tenemos nuesto dataframe listo para mandar al preprocesamiento. Como se observó, a los datos se les hizo una primera limpieza, la razón de esto fue para que en el preprocesamiento se hicieran los pasos fundamentales y no pasos extras, como fue este caso de quitar ciertos patrones que estorbaban.